# 1.5_data_modeling_cassandra_project
<img src="https://upload.wikimedia.org/wikipedia/commons/5/5e/Cassandra_logo.svg" width="100" height="100">

# 1. ETL pipeline for pre-processing the files

In [ ]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

# SKIP TO PART II because the following cells only work in the Udacity Workspace environment.

#### Creating list of filepaths to process original event csv data files

In [ ]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [ ]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [ ]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

# 2. Connection and set up

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the Part I code is run:<br>

<img src="1.5_data_modeling_cassandra_project.jpg" width="50%">

The file contains 11 columns, which will in this chapter be referred to by their order numbers, left to right:
0. artist 
1. firstName
2. gender
3. itemInSession
4. lastName
5. length
6. level
7. location
8. sessionId
9. song
10. userId


In [ ]:
# Connect to the cluster
try: 
    cluster = Cluster(['127.0.0.1']) # If running locally
    session = cluster.connect()
except Exception as e:
    print(e)

# Create Keyspace
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS udacity 
        WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    """)
except Exception as e:
    print(e)

# Set keyspace
try:
    session.set_keyspace("udacity")
except Exception as e:
    print(e)

# 3. Queries

1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    
3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


To answer these 3 requests, we need 3 tables, each with its own PRIMARY KEY: 
- query 1, table artist_song_length, 
    - PRIMARY KEY(sessionId, itemInSession)
        - partition key = sessionId 
            - for even distribution of sessions across nodes, because judging by the sessionId=338, there are hundreds of them,
            - for efficient retrieval, because sessionId is the primary filter in this query
        - clustering column = itemInSession 
            - to sort items within a session, because each sessionId may contain multiple items.
---
- query 2, table artist_song_user, 
    - PRIMARY KEY(userid, sessionId, itemInSession)
        - partition key = userid, again for even distribution and fast query (WHERE userid=10).
        - clustering columns = sessionId, itemInSession, 
            - sessionId, to filter users by sessions, 
            - itemInSession, because each song/item in a session is assigned its playing order number, and the query asks for the songs in the output to be sorted by that number = itemInSession.
---
- query 3, table user_song, 
    - PRIMARY KEY(song, userId)
        - partition key = song, for even distribution, because every user has 1 or more sessions, and each session had 1 or more songs.
        - clustering column = userId, to create a unique identifier for each listener of the song.


# Query 1  
Get [artist, song, length] for [sessionId=338, itemInSession=4]

In [ ]:
# CREATE TABLE
query = "CREATE TABLE artist_song_length "
query = query + "(sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY(sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

# INSERT INTO
file = 'event_data_file_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header

    for line in csvreader:
        query = "INSERT INTO artist_song_length (sessionId, itemInSession, artist, song, length) "
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

# QUERY
query = "SELECT artist, song, length FROM artist_song_length WHERE sessionId=338 AND itemInSession=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song, row.length)

# Query 2 
Get [artist, song, firstName, lastName] for [userid=10, sessionId=182]

In [ ]:
# CREATE TABLE
query = "CREATE TABLE artist_song_user "
query = query + "(userid int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY(userid, sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
# INSERT INTO
file = 'event_data_file_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_song_user (userid, sessionId, itemInSession, artist, song, firstName, lastName) "
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

# QUERY
query = "SELECT artist, song, firstname, lastname FROM artist_song_user WHERE userid=10 AND sessionid=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song, row.firstname, row.lastname)

# Query 3  
Get [firstName, lastName] for [song='All Hands Against His Own']

In [ ]:
# CREATE TABLE
query = "CREATE TABLE user_song "
query = query + "(song text, userId int, firstName text, lastName text, PRIMARY KEY(song, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
# INSERT INTO
file = 'event_data_file_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header

    for line in csvreader:
        query = "INSERT INTO user_song (song, userId, firstName, lastName) "
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

# QUERY
query = "SELECT firstname, lastname FROM user_song WHERE song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.firstname, row.lastname)          

In [ ]:
# DROP TABLES
for t in ["artist_song_length", "artist_song_user", "user_song"]:
    query = f"DROP TABLE {t}"
    try:
        rows = session.execute(query)
    except Exception as e:
        print(e)

# Close the connection
cluster.shutdown()
session.shutdown()